In [1]:
import pandas as pd
import numpy as np
import os
import datetime

from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
from sklearn.preprocessing import MinMaxScaler


Using TensorFlow backend.
C:\Users\mpich\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\mpich\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\mpich\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\mpich\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:
df = pd.read_csv("manipulated_pun.csv", sep=";", decimal=',', header='infer')

df.head()

,Data,Ora,PUN
0,20170101,1,53.30
1,20170101,2,52.00
2,20170101,3,51.00
3,20170101,4,47.27
4,20170101,5,45.49


In [3]:
df["Data"] = df["Data"].apply(lambda x: str(x))
df["Data"] = df["Data"].apply(lambda x: datetime.datetime.strptime(x,"%Y%m%d"))
df = df.loc[df['Data'] <= '2020-01-01']

In [4]:
df["Ora2"] = pd.to_datetime(df.Ora, unit="h").dt.strftime("%H:%M")
df["Datetime"] = df["Data"].dt.strftime("%Y-%m-%d") + " " + df["Ora2"]
df.index = pd.DatetimeIndex(df.Datetime)

In [5]:
df.head()

,Data,Ora,PUN,Ora2,Datetime
Datetime,,,,,
2017-01-01 01:00:00,2017-01-01,1,53.30,01:00,2017-01-01 01:00
2017-01-01 02:00:00,2017-01-01,2,52.00,02:00,2017-01-01 02:00
2017-01-01 03:00:00,2017-01-01,3,51.00,03:00,2017-01-01 03:00
2017-01-01 04:00:00,2017-01-01,4,47.27,04:00,2017-01-01 04:00
2017-01-01 05:00:00,2017-01-01,5,45.49,05:00,2017-01-01 05:00


In [6]:
target = df["PUN"]
df["df24"] = target.shift(24)
target = df["PUN"].iloc[25:]
features = df[["PUN", "df24"]].iloc[25:]

In [7]:
mask_test = df["Datetime"] > "2019-12-25"
mask_train = df["Datetime"] <= "2019-12-25"

# %%
test = df.loc[mask_test]
test

# %%
train = df.loc[mask_train]
train



,Data,Ora,PUN,Ora2,Datetime,df24
Datetime,,,,,,
2017-01-01 01:00:00,2017-01-01,1,53.30,01:00,2017-01-01 01:00,NaN
2017-01-01 02:00:00,2017-01-01,2,52.00,02:00,2017-01-01 02:00,NaN
2017-01-01 03:00:00,2017-01-01,3,51.00,03:00,2017-01-01 03:00,NaN
2017-01-01 04:00:00,2017-01-01,4,47.27,04:00,2017-01-01 04:00,NaN
2017-01-01 05:00:00,2017-01-01,5,45.49,05:00,2017-01-01 05:00,NaN
...,...,...,...,...,...,...
2019-12-24 20:00:00,2019-12-24,20,47.43,20:00,2019-12-24 20:00,44.40
2019-12-24 21:00:00,2019-12-24,21,45.70,21:00,2019-12-24 21:00,40.99
2019-12-24 22:00:00,2019-12-24,22,36.53,22:00,2019-12-24 22:00,36.90


In [8]:
trainX = train['df24'][24:].values
testX = test['df24'].values
trainY = train['PUN'][24:].values
testY = test["PUN"].values

trainY


# %%
testY.shape


# %%
trainX.shape = (26088, 1)
testX.shape = (192, 1) 
trainY.shape = (26088, 1)
testY.shape = (192, 1)



In [9]:
def data():
    
    df = pd.read_csv("manipulated_pun.csv", sep=";", decimal=',', header='infer')
    
    df["Data"] = df["Data"].apply(lambda x: str(x))
    df["Data"] = df["Data"].apply(lambda x: datetime.datetime.strptime(x,"%Y%m%d"))
    df = df.loc[df['Data'] <= '2020-01-01']
    
    df["Ora2"] = pd.to_datetime(df.Ora, unit="h").dt.strftime("%H:%M")
    df["Datetime"] = df["Data"].dt.strftime("%Y-%m-%d") + " " + df["Ora2"]
    df.index = pd.DatetimeIndex(df.Datetime)

    target = df["PUN"]
    df["df24"] = target.shift(24)
    target = df["PUN"].iloc[25:]
    features = df[["PUN", "df24"]].iloc[25:]
    
    mask_test = df["Datetime"] > "2019-12-25"
    mask_train = df["Datetime"] <= "2019-12-25"

    # %%
    test = df.loc[mask_test]
    test

    # %%
    train = df.loc[mask_train]
    train
    
    
    
    
    trainX = train['df24'][24:].values
    testX = test['df24'].values
    trainY = train['PUN'][24:].values
    testY = test["PUN"].values

    trainY


    # %%
    testY.shape


    # %%
    trainX.shape = (26088, 1)
    testX.shape = (192, 1) 
    trainY.shape = (26088, 1)
    testY.shape = (192, 1)
    
    
    scaler = MinMaxScaler(feature_range=(0,1))
    X_train = scaler.fit_transform(trainX)
    X_test = scaler.fit_transform(testX)
    y_train = scaler.fit_transform(trainY)
    y_test = scaler.fit_transform(testY)
    
    return X_train, X_test, y_train, y_test



In [10]:
def create_model(X_train, y_train, X_test, y_test):
    import tensorflow.python.keras
    from tensorflow.python.keras.models import Sequential
    from tensorflow.python.keras.layers import Dense, Activation, Dropout
    from tensorflow.python.keras import backend as K

    from tensorflow.python.keras import utils, layers, Sequential
    
    model = Sequential()
    # the model will take as input arrays of shape (*, X_train.shape[1])
    # and output arrays of shape (*, 25)
    model.add(Dense(64, input_shape=(X_train.shape[1],)))
    model.add(Activation('relu'))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(Dense({{choice([64, 128, 256, 512, 1024, 2048])}}))
    model.add(Activation({{choice(['relu', 'sigmoid'])}}))
    model.add(Dropout({{uniform(0, 1)}}))

    model.add(Dense(1))
    model.add(Activation('softmax'))

    model.compile(loss='mse', metrics=['mean_absolute_error'],
                  optimizer={{choice(['rmsprop', 'adam', 'sgd'])}})

    result = model.fit(X_train, y_train,
              batch_size={{choice([22, 64, 128, 256, 512, 1024, 2048])}},
              epochs=10,
              verbose=2,
              validation_split=0.15)
    #get the highest validation mae of the training epochs
    validation_mae = np.amax(result.history['mean_absolute_error']) 
    print('Best validation mae of epoch:', validation_mae)
    return {'loss': validation_mae, 'status': STATUS_OK, 'model': model}


In [11]:
best_run, best_model = optim.minimize(model=create_model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=5,
                                      trials=Trials(),
                                      notebook_name='Hyperparameters_opt')

>>> Imports:
#coding=utf-8

try:
    import pandas as pd
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import os
except:
    pass

try:
    import datetime
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    from sklearn.preprocessing import MinMaxScaler
except:
    pass

try:
    import tensorflow.python.keras
except:
    pass

try:
    from tensorflow.python.keras.models import Sequential
except:
    pass

try:
    from tensorflow.python.keras.layers import Dense, Activation, Dropout
except:
    pass

try:
    from tensorflow.python.keras import backend as K
except:
    pass

try:
    from tensorflow.python.keras import utils, layers, Sequential
except:
    pass

>>> Hyperas search space:

def get_space():
    return {
        'Dropout': hp.uniform('Dropout', 0, 1),
        'Dense': h

Epoch 6/10                                                                     
 - 0s - loss: 0.4547 - mean_absolute_error: 0.6681 - val_loss: 0.5108 - val_mean_absolute_error: 0.7112

Epoch 7/10                                                                     
 - 0s - loss: 0.4547 - mean_absolute_error: 0.6681 - val_loss: 0.5108 - val_mean_absolute_error: 0.7112

Epoch 8/10                                                                     
 - 0s - loss: 0.4547 - mean_absolute_error: 0.6681 - val_loss: 0.5108 - val_mean_absolute_error: 0.7112

Epoch 9/10                                                                     
 - 0s - loss: 0.4547 - mean_absolute_error: 0.6681 - val_loss: 0.5108 - val_mean_absolute_error: 0.7112

Epoch 10/10                                                                    
 - 0s - loss: 0.4547 - mean_absolute_error: 0.6681 - val_loss: 0.5108 - val_mean_absolute_error: 0.7112

Best validation mae of epoch:                                              

In [12]:
print("Best performing model chosen hyper-parameters:")
print(best_run)

Best performing model chosen hyper-parameters:
{'Activation': 0, 'Dense': 2, 'Dropout': 0.8366666847115819, 'Dropout_1': 0.9128294469805703, 'batch_size': 6, 'optimizer': 0}
